<a href="https://colab.research.google.com/github/ambideXtrous9/Seq2Seq-Attention-QA/blob/main/Seq2Seq_with_NoAttentionQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet transformers
!pip install --quiet pytorch-lightning
!pip install --quiet tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
torch.set_float32_matmul_precision('high')
import random
import numpy as np
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer as Tokenizer,AutoModelForSeq2SeqLM
from torch.optim import AdamW

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
MODEL_NAME = 'google/flan-t5-small'

In [ ]:
tokenizer = Tokenizer.from_pretrained(MODEL_NAME)

In [ ]:
path = '/content/drive/MyDrive/MTP CODE/NewsQA_SPAN.feather'

In [ ]:
df = pd.read_feather(path)

In [ ]:
df = df.iloc[:5000]

In [ ]:
train_df, val_df = train_test_split(df,test_size=0.2)
val_df, test_df = train_test_split(val_df,test_size=0.5)

In [ ]:
class QADataset(Dataset):
  def __init__(self,data : pd.DataFrame,tokenizer : Tokenizer,source_max_token_len : int = 200,target_max_token_len : int = 15):

    self.tokenizer = tokenizer
    self.data = data
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def __len__(self):
    return len(self.data)
  
  def __getitem__(self,index : int):
    data_row = self.data.iloc[index]

    source_encoding = tokenizer(
        data_row['question'],
        data_row['paragraph'],
        max_length = self.source_max_token_len,
        padding = "max_length",
        truncation = "only_second",
        return_attention_mask = True,
        add_special_tokens = True,
        return_tensors = "pt")
    
    target_encoding = tokenizer(
        data_row['answer'],
        max_length = self.target_max_token_len,
        padding = "max_length",
        truncation = True,
        return_attention_mask = True,
        add_special_tokens = True,
        return_tensors = "pt")
    
    labels = target_encoding["input_ids"]
    #labels[labels == 0] = -100

    return dict(
        answer = data_row['answer'],
        input_ids = source_encoding['input_ids'].flatten(),
        attention_mask = source_encoding['attention_mask'].flatten(),
        labels = labels.flatten())


In [ ]:
sample_dataset = QADataset(df,tokenizer)

In [ ]:
tokenizer.vocab_size

In [ ]:
for data in sample_dataset:
  print(type(data['input_ids']))
  print(data['input_ids'])
  break

In [ ]:
class QADataModule(pl.LightningDataModule):
  def __init__(self,train_df , val_df, test_df,tokenizer : Tokenizer,batch_size : int = 8,source_max_token_len : int = 200,target_max_token_len : int = 15):
    super().__init__()
    self.batch_size = batch_size
    self.train_df = train_df
    self.test_df = test_df
    self.val_df = val_df
    self.tokenizer = tokenizer
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def setup(self,stage=None):
    self.train_dataset = QADataset(self.train_df,self.tokenizer,self.source_max_token_len,self.target_max_token_len)
    self.val_dataset = QADataset(self.val_df,self.tokenizer,self.source_max_token_len,self.target_max_token_len)
    self.test_dataset = QADataset(self.test_df,self.tokenizer,self.source_max_token_len,self.target_max_token_len)
    

  def train_dataloader(self):
    return DataLoader(self.train_dataset,batch_size = self.batch_size,shuffle=True,num_workers=2)

  def val_dataloader(self):
    return DataLoader(self.val_dataset,batch_size = self.batch_size,num_workers=2)

  def test_dataloader(self):
    return DataLoader(self.test_dataset,batch_size = self.batch_size,num_workers=2)   

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropout)
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers, dropout=dropout, batch_first=True)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        outputs, hidden = self.gru(embedded)
        return outputs, hidden




In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        max_len = encoder_outputs.size(1)
        h = hidden.repeat(max_len, 1, 1).transpose(0, 1)
        encoder_outputs = encoder_outputs.transpose(1, 2)
        hidden_linear = hidden[-1].unsqueeze(0).unsqueeze(2)  # add extra dim for bmm
        hidden_linear = self.attn(hidden_linear.transpose(1, 2)).transpose(1, 2)  # transpose for correct shape
        attn_energies = self.v(torch.tanh(hidden_linear + encoder_outputs))
        attn_weights = F.softmax(attn_energies, dim=1)
        context = torch.bmm(attn_weights.transpose(1,2), encoder_outputs)
        return context


In [ ]:
class AttentionDecoder(nn.Module):
    def __init__(self, output_size, hidden_size, num_layers, dropout):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropout)
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hidden_size, output_size)
        self.attention = Attention(hidden_size)

    def forward(self, x, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(x))
        context, _ = self.attention(hidden, encoder_outputs)
        rnn_input = torch.cat((embedded, context), dim=2)
        output, hidden = self.gru(rnn_input, hidden)
        output = self.fc_out(output)
        return output, hidden


In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_size, hidden_size, num_layers, dropout):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropout)
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hidden_size, output_size)
        self.attention = Attention(hidden_size)

    def forward(self, x, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(x))
        #context = self.attention(hidden[-1], encoder_outputs)
        #rnn_input = torch.cat((embedded, context), dim=2)
        output, hidden = self.gru(embedded, hidden)
        output = self.fc_out(output)
        return output, hidden

In [ ]:
class Seq2Seq(pl.LightningModule):
    def __init__(self, encoder, decoder, pad_idx):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.pad_idx = pad_idx
        self.trained_model = NQAModel.load_from_checkpoint(cppath)
        self.trained_model.freeze()

    def forward(self, src, attn,trg):
        batch_size = src.size(0)
        max_len = trg.size(1)
        trg_vocab_size = self.decoder.fc_out.out_features

        encoder_outputs, hidden = self.encoder(src)


        outputs = torch.zeros(batch_size, max_len, trg_vocab_size).to(self.device)
        output = trg[:, 0]
        for t in range(1, max_len):
            output, hidden = self.decoder(output.unsqueeze(1), hidden, encoder_outputs)
            outputs[:, t, :] = output.squeeze(1)
            top1 = output.argmax(2)
            output = top1.squeeze(1)
        
        return outputs, hidden

    def training_step(self, batch, batch_idx):
        
        src = batch['input_ids']
        attn = batch['attention_mask']
        trg = batch['labels']

        trg_input = trg
        trg_output = trg

        output, hidden = self(src, attn,trg_input)


        output = output.reshape(-1, output.shape[-1])
        trg_output = trg_output.reshape(-1)

        train_loss = F.cross_entropy(output, trg_output, ignore_index=self.pad_idx)

        self.log_dict({"train_loss" : train_loss,
                       },prog_bar=True,logger=True)

        return train_loss

    def validation_step(self, batch, batch_idx):
        src = batch['input_ids']
        attn = batch['attention_mask']
        trg = batch['labels']

        trg_input = trg
        trg_output = trg

        output, hidden = self(src, attn,trg_input)


        

        output = output.reshape(-1, output.shape[-1])
        trg_output = trg_output.reshape(-1)

        val_loss = F.cross_entropy(output, trg_output, ignore_index=self.pad_idx)

        self.log_dict({"val_loss" : val_loss
                       },prog_bar=True,logger=True)


        return val_loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer


In [ ]:
BATCH_SIZE = 64
N_EPOCHS = 50

data_module = QADataModule(train_df,val_df,test_df,tokenizer,batch_size = BATCH_SIZE)
data_module.setup()

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath = 'checkpoints',
    filename = 'Seq2Seq',
    save_top_k = 1,
    verbose = True,
    monitor = 'val_loss',
    mode = 'min'
)

In [ ]:
encoder = Encoder(input_size=tokenizer.vocab_size, hidden_size=512, num_layers=1, dropout=0.2)

In [ ]:
decoder = Decoder(output_size=tokenizer.vocab_size, hidden_size=512, num_layers=1, dropout=0.2)

In [ ]:
model = Seq2Seq(encoder,decoder,pad_idx=0)

In [ ]:
trainer = pl.Trainer(devices=-1, accelerator="gpu",
    callbacks=[checkpoint_callback],
    max_epochs = N_EPOCHS
)

In [ ]:
trainer.fit(model,data_module)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
def predict(question):
    model.eval()

    # Tokenize the source text
    source_tokens = tokenizer(
        question['question'],
        question['paragraph'],
        max_length=200,
        padding="max_length",
        truncation="only_second",
        add_special_tokens=True,
        return_tensors="pt")['input_ids'].flatten().to(device)

    # Reshape the source tokens to match the expected input shape of the encoder
    source_tokens = source_tokens.unsqueeze(0).to(device)

    model.to(device)

    # Encode the source text
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(source_tokens)

    max_length = 15
    # Initialize the predicted sentence
    outputs = [0]

    # Generate the output sequence token by token
    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).unsqueeze(0).to(device)

        # Decode the next token
        with torch.no_grad():
            output, hidden = model.decoder(previous_word, hidden, encoder_outputs)
            best_guess = output.argmax(2).item()

        # Add the predicted token to the predicted sentence
        outputs.append(best_guess)

        # If the predicted token is the end-of-sequence token, stop generating further tokens
        if best_guess == tokenizer.sep_token_id:
            break

    # Convert the predicted sentence back to text
    
    print(outputs)
    predicted_text = tokenizer.decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return predicted_text


In [ ]:
sample_question = test_df.iloc[10]


In [ ]:
sample_question['question']


In [ ]:
sample_question['answer']


In [ ]:
predict(sample_question)